In [1]:
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import random as rand
import numpy as np
import math

In [2]:
random = 24

In [3]:
data = pd.read_csv('games.csv')
random = 24
data = data[["Metacritic", "Indie", "Presence", "Platform", 
             "Memory", "RatingsBreakdown", "ReleaseDate", "Soundtrack", 
             "OriginalCost", "DiscountedCost", "Controller",
             "Achievements", "Languages", "Storage", "Tags"]]

In [4]:
data.describe(include= 'all')

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
count,3356.000000,30045.000000,3.015600e+04,30123,28316,15044,27024,30045.000000,29504,727,29976.000000,30156.000000,30027,27491,30045
unique,NaN,NaN,NaN,2041,699,4806,4133,NaN,396,121,NaN,NaN,3196,2067,24866
top,NaN,NaN,NaN,PC,2 GB RAM,meh: 1,2018-08-22,NaN,$9.99,$0.49,NaN,NaN,English,1 GB available space,"Adventure, Casual, Hidden Object"
freq,NaN,NaN,NaN,15720,6487,910,238,NaN,4144,260,NaN,NaN,17133,2303,258
mean,72.924911,0.740722,8.822923e+04,NaN,NaN,NaN,NaN,0.004760,NaN,NaN,0.457666,41.047984,NaN,NaN,NaN
std,10.805296,0.438246,2.447597e+05,NaN,NaN,NaN,NaN,0.068826,NaN,NaN,0.498213,316.231316,NaN,NaN,NaN
min,20.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,67.000000,0.000000,1.890000e+02,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
50%,74.000000,1.000000,3.380000e+02,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
75%,80.000000,1.000000,4.981500e+03,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000,21.000000,NaN,NaN,NaN


In [5]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

Metacritic
[83. 82. 90. 71. 68. nan 75. 86. 72. 69. 97. 85. 88. 70. 89. 73. 93. 81.
 78. 67. 80. 74. 92. 51. 79. 91. 87. 77. 84. 76. 65. 95. 94. 49. 62. 66.
 59. 57. 60. 64. 61. 54. 63. 96. 58. 45. 43. 44. 48. 52. 55. 56. 53. 33.
 38. 29. 46. 47. 34. 50. 40. 32. 35. 41. 37. 27. 42. 39. 24. 30. 36. 20.]
Indie
[ 0.  1. nan]
Presence
[1009588. 1007425. 1009306. ...  196163.  226332.   45207.]
Platform
['PC, Xbox 360, PlayStation 3'
 'PlayStation 5, Web, Xbox Series X, PC, Xbox One, PlayStation 4'
 'Linux, macOS, PC' ... 'PlayStation 3, macOS, PC' 'Wii'
 'PlayStation 4, Wii U']
Memory
['2 GB RAM' '6 GB RAM' '4 GB RAM' '3 GB RAM' '8 GB RAM' '10 GB RAM' nan
 '12 GB RAM' '512 MB RAM' '5 GB RAM' '2.5GB ' '4 GB RAM ' '1 GB RAM'
 '4 GB RAM DDR3 ' '4096 MB RAM' '2GB RAM  ' '4 GB or higher ' '2GB RAM '
 '1024 MB RAM' '8GB RAM' '6 GB RAM (64-bit)'
 '2 GB for 32-bit OS or 4 GB for 64-bit OS' '6 GB RAM ' '2GB '
 '(XP) 1 GB; (Vista) 1.5 GB ' '2 GB(XP)/ 2 GB(Vista) ' '512 MB '
 '512 MB RAM (768 MB RAM 

In [6]:
actions = {k:[] for k in data.columns}
actions['Indie'].append(lambda x: x == 1.0) 
actions['Controller'].append(lambda x: x == 1.0) 
actions['Soundtrack'].append(lambda x: x == 1.0)
actions['ReleaseDate'].append(lambda x: int(x.split('-')[0]))

def replace_MB(x):
    x = str(x).lower()
    first = re.search(r"\d", x)
    if (first == None):
        return None
    first = first.start()
    i = first+1
    while(i<len(x) and (x[i].isdigit() or x[i] == '.')):
        i = i+1
    last = i
    while(i<len(x) and x[i:i+2] != "gb" and x[i:i+2] != "mb"):
        i = i+1
    if (i == len(x) or x[i:i+2] == "gb"):
        return(float(x[first:last])*1000)
    return(float(x[first:last]))


def clean_originalCost(x):
    x = str(x).replace('$', '')
    try:
        number = float(x)
        return number
    except:
        return 0


actions['OriginalCost'].append(lambda x: clean_originalCost(x))
actions['OriginalCost'].append(lambda x: float(str(x).replace('$', '')))
actions['DiscountedCost'].append(lambda x: float(str(x).replace('$', '')))
actions['Storage'].append(lambda x: replace_MB(x))
actions['Memory'].append(lambda x: replace_MB(x))

for name in actions:
    for action in actions[name]:
        data.loc[:,name] = data[data[name].notna()][name].apply(action)

data['DiscountedCost'] = data['DiscountedCost'].fillna(0)
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
0,83.0,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012.0,False,0.00,0.0,True,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15000.0,"FPS, Shooter, Multiplayer, Competitive, Action..."
1,82.0,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017.0,False,0.00,0.0,True,61.0,"English, French, Italian, German, Japanese, Ko...",105000.0,"Free to Play, Looter Shooter, FPS, Multiplayer..."
2,90.0,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013.0,False,0.00,0.0,True,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15000.0,"Free to Play, MOBA, Multiplayer, Strategy, eSp..."
3,71.0,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014.0,False,19.99,0.0,True,0.0,"English, French, German, Russian",85000.0,"RPG, MMORPG, Open World, Massively Multiplayer..."
4,68.0,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018.0,False,39.99,0.0,True,308.0,"English, French, Italian, German, Russian",50000.0,"Adventure, Action, Pirates, Open World, Multip..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018.0,False,0.99,0.0,True,0.0,English,10.0,"Indie, Casual"
30246,NaN,True,13.0,PC,6000.0,NaN,2017.0,False,0.99,0.0,False,0.0,English,2000.0,"Indie, Puzzle, Pixel Graphics, Mystery, Experi..."
30247,NaN,False,11.0,PC,2048.0,NaN,2018.0,False,1.99,0.0,False,0.0,English,2048.0,Casual
30248,NaN,True,423.0,PC,500.0,NaN,2017.0,False,0.99,0.0,True,0.0,English,300.0,"Action, Adventure, Indie, Casual"


In [7]:
platform = pd.Series(data['Platform']).str.get_dummies(', ')
platform.columns
data['Platform-Mobile'] = platform[['Android','iOS']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-PC'] = platform[['Apple II','Atari ST', 'Classic Macintosh', 'Commodore / Amiga',  'Linux', 'PC', 'Web', 'macOS']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Xbox'] = platform[['Xbox', 'Xbox 360', 'Xbox One']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-PlayStation'] = platform[['PS Vita', 'PSP', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'PlayStation 4']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Nintendo'] = platform[['Game Boy', 'GameCube', 'Nintendo 3DS', 'Nintendo DS', 'Nintendo Switch', 'Wii', 'Wii U']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-Sega'] = platform[['Dreamcast','Game Gear','Genesis','SEGA Master System', 'SEGA Saturn']].sum(axis = 1).apply(lambda x: x > 0) 
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,...,Achievements,Languages,Storage,Tags,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega
0,83.0,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012.0,False,0.00,0.0,...,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15000.0,"FPS, Shooter, Multiplayer, Competitive, Action...",False,True,True,True,False,False
1,82.0,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017.0,False,0.00,0.0,...,61.0,"English, French, Italian, German, Japanese, Ko...",105000.0,"Free to Play, Looter Shooter, FPS, Multiplayer...",False,True,True,True,False,False
2,90.0,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013.0,False,0.00,0.0,...,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15000.0,"Free to Play, MOBA, Multiplayer, Strategy, eSp...",False,True,False,False,False,False
3,71.0,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014.0,False,19.99,0.0,...,0.0,"English, French, German, Russian",85000.0,"RPG, MMORPG, Open World, Massively Multiplayer...",False,True,False,False,False,False
4,68.0,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018.0,False,39.99,0.0,...,308.0,"English, French, Italian, German, Russian",50000.0,"Adventure, Action, Pirates, Open World, Multip...",False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018.0,False,0.99,0.0,...,0.0,English,10.0,"Indie, Casual",False,True,False,False,False,False
30246,NaN,True,13.0,PC,6000.0,NaN,2017.0,False,0.99,0.0,...,0.0,English,2000.0,"Indie, Puzzle, Pixel Graphics, Mystery, Experi...",False,True,False,False,False,False
30247,NaN,False,11.0,PC,2048.0,NaN,2018.0,False,1.99,0.0,...,0.0,English,2048.0,Casual,False,True,False,False,False,False
30248,NaN,True,423.0,PC,500.0,NaN,2017.0,False,0.99,0.0,...,0.0,English,300.0,"Action, Adventure, Indie, Casual",False,True,False,False,False,False


In [8]:
def RatingsBreakdown(x):
    x = x.split(', ')
    result = [np.nan, np.nan, np.nan, np.nan]
    for i in x:
        if i.split(' ')[0] == 'recommended:':
            result[0] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'meh:':
            result[1] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'exceptional:':
            result[2] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'skip:':
            result[3] = float(i.split(' ')[1])
    return result  
        
df_aux = pd.DataFrame(data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].apply(lambda x: RatingsBreakdown(x)))
df_aux = pd.DataFrame(df_aux.RatingsBreakdown.values.tolist(),index = data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].index, columns=['RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip'])
data = data.join(df_aux)
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,...,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip
0,83.0,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012.0,False,0.00,0.0,...,False,True,True,True,False,False,882.0,501.0,283.0,185.0
1,82.0,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017.0,False,0.00,0.0,...,False,True,True,True,False,False,726.0,465.0,227.0,106.0
2,90.0,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013.0,False,0.00,0.0,...,False,True,False,False,False,False,446.0,496.0,228.0,371.0
3,71.0,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014.0,False,19.99,0.0,...,False,True,False,False,False,False,21.0,12.0,5.0,1.0
4,68.0,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018.0,False,39.99,0.0,...,False,True,True,False,False,False,198.0,131.0,62.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018.0,False,0.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30246,NaN,True,13.0,PC,6000.0,NaN,2017.0,False,0.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30247,NaN,False,11.0,PC,2048.0,NaN,2018.0,False,1.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30248,NaN,True,423.0,PC,500.0,NaN,2017.0,False,0.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN


In [9]:
Tags = pd.Series(data['Tags']).str.get_dummies(', ').apply(lambda x: x > 0)
del Tags['Indie']
del Tags['Soundtrack']
del Tags['Controller']
Languages = pd.Series(data['Languages']).str.get_dummies(', ').apply(lambda x: x > 0)


data = data.join(Tags)
data = data.join(Languages)

In [10]:
del data['RatingsBreakdown']
del data['Platform']
del data['Tags']
del data['Languages']

In [11]:
for name in data.columns:
    print(name, data[name].isna().sum())

Metacritic 26894
Indie 205
Presence 94
Memory 1945
ReleaseDate 3226
Soundtrack 205
OriginalCost 746
DiscountedCost 0
Controller 274
Achievements 94
Storage 2762
Platform-Mobile 0
Platform-PC 0
Platform-Xbox 0
Platform-PlayStation 0
Platform-Nintendo 0
Platform-Sega 0
RatingsBreakdown-Recommended 20274
RatingsBreakdown-Meh 19575
RatingsBreakdown-Exceptional 22267
RatingsBreakdown-Skip 20356
1980s 0
1990's 0
2.5D 0
2D 0
2D Fighter 0
2D Platformer 0
360 Video 0
3D 0
3D Fighter 0
3D Platformer 0
3D Vision 0
4 Player Local 0
4X 0
6DOF 0
ATV 0
Abstract 0
Action 0
Action RPG 0
Action RTS 0
Action Roguelike 0
Action-Adventure 0
Addictive 0
Adventure 0
Agriculture 0
Aliens 0
Alternate History 0
Ambient 0
America 0
Animation & Modeling 0
Anime 0
Arcade 0
Archery 0
Arena Shooter 0
Artificial Intelligence 0
Assassin 0
Asymmetric VR 0
Asynchronous Multiplayer 0
Atmospheric 0
Audio Production 0
Auto Battler 0
Automation 0
Automobile Sim 0
BMX 0
Base Building 0
Baseball 0
Based On A Novel 0
Basketbal

In [12]:
for col in data.columns:
    if len(data[col].unique()) == 1:
        data.drop(col,inplace=True,axis=1)

In [13]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

Metacritic
[83. 82. 90. 71. 68. nan 75. 86. 72. 69. 97. 85. 88. 70. 89. 73. 93. 81.
 78. 67. 80. 74. 92. 51. 79. 91. 87. 77. 84. 76. 65. 95. 94. 49. 62. 66.
 59. 57. 60. 64. 61. 54. 63. 96. 58. 45. 43. 44. 48. 52. 55. 56. 53. 33.
 38. 29. 46. 47. 34. 50. 40. 32. 35. 41. 37. 27. 42. 39. 24. 30. 36. 20.]
Indie
[False True nan]
Presence
[1009588. 1007425. 1009306. ...  196163.  226332.   45207.]
Memory
[2.0000000e+03 6.0000000e+03 4.0000000e+03 3.0000000e+03 8.0000000e+03
 1.0000000e+04           nan 1.2000000e+04 5.1200000e+02 5.0000000e+03
 2.5000000e+03 1.0000000e+03 4.0960000e+03 1.0240000e+03 2.0480000e+03
 2.5600000e+02 1.5000000e+03 3.0720000e+03 1.5600000e+02 2.5600000e+05
 6.4000000e+01 1.6000000e+04 1.2500000e+02 7.6800000e+02 4.0000000e+00
 1.2800000e+02 9.6000000e+01 2.0000000e+00 1.6000000e+01 5.0000000e+02
 3.4000000e+02 5.1300000e+02 3.4900000e+02 6.1440000e+03 4.0000000e+02
 3.8400000e+02 1.2000000e+03 3.2000000e+01 9.4000000e+01 8.1920000e+03
 6.0000000e+02 2.5600000e+03 

In [14]:
from scipy import stats

In [15]:
col =['Presence', 'Memory', 'ReleaseDate', 'Achievements', 'Storage']
out = None
for name in col:
    data = data[~(data[name] - data[name].mean() > 3*data[name].std())]

In [16]:
for name in data.columns:
    print(name)

Metacritic
Indie
Presence
Memory
ReleaseDate
Soundtrack
OriginalCost
DiscountedCost
Controller
Achievements
Storage
Platform-Mobile
Platform-PC
Platform-Xbox
Platform-PlayStation
Platform-Nintendo
Platform-Sega
RatingsBreakdown-Recommended
RatingsBreakdown-Meh
RatingsBreakdown-Exceptional
RatingsBreakdown-Skip
1980s
1990's
2.5D
2D
2D Fighter
2D Platformer
360 Video
3D
3D Fighter
3D Platformer
3D Vision
4 Player Local
4X
6DOF
ATV
Abstract
Action
Action RPG
Action RTS
Action Roguelike
Action-Adventure
Addictive
Adventure
Agriculture
Aliens
Alternate History
Ambient
America
Animation & Modeling
Anime
Arcade
Archery
Arena Shooter
Artificial Intelligence
Assassin
Asymmetric VR
Asynchronous Multiplayer
Atmospheric
Audio Production
Auto Battler
Automation
Automobile Sim
BMX
Base Building
Baseball
Based On A Novel
Basketball
Batman
Battle Royale
Beat 'em up
Beautiful
Benchmark
Bikes
Blood
Board Game
Bowling
Boxing
Breathtaking
Building
Bullet Hell
Bullet Time
CRPG
Capitalism
Card Battler
Card 

In [17]:
labels_num = ['Presence','Memory','ReleaseDate','OriginalCost','DiscountedCost','Achievements','Storage',
                'RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip',
              'Metacritic','Controller', 'Indie', 'Soundtrack']
dataNum = data[labels_num]
dataNoNum = data[data.columns.difference(labels_num)]
dataNum

,Presence,Memory,ReleaseDate,OriginalCost,DiscountedCost,Achievements,Storage,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip,Metacritic,Controller,Indie,Soundtrack
4,777456.0,4000.0,2018.0,39.99,0.00,308.0,50000.0,198.0,131.0,62.0,39.0,68.0,True,False,False
5,158804.0,8000.0,2019.0,29.99,0.00,0.0,15000.0,18.0,4.0,21.0,NaN,NaN,False,True,False
12,634023.0,4000.0,2016.0,59.99,0.00,445.0,12000.0,393.0,70.0,219.0,14.0,88.0,False,False,False
18,185610.0,12000.0,2020.0,59.99,44.99,16.0,NaN,16.0,7.0,54.0,4.0,93.0,True,False,False
20,130138.0,6000.0,NaN,29.99,0.00,90.0,3000.0,60.0,6.0,29.0,2.0,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,105.0,512.0,2018.0,0.99,0.00,0.0,10.0,NaN,NaN,NaN,NaN,NaN,True,True,False
30246,13.0,6000.0,2017.0,0.99,0.00,0.0,2000.0,NaN,NaN,NaN,NaN,NaN,False,True,False
30247,11.0,2048.0,2018.0,1.99,0.00,0.0,2048.0,NaN,NaN,NaN,NaN,NaN,False,False,False
30248,423.0,500.0,2017.0,0.99,0.00,0.0,300.0,NaN,NaN,NaN,NaN,NaN,True,True,False


In [18]:
min_max_scaler = MinMaxScaler()
data_scaled = min_max_scaler.fit_transform(dataNum.values)
data_imputed = KNNImputer(n_neighbors=5,weights='distance').fit_transform(pd.DataFrame(data_scaled))


data1 = pd.DataFrame(min_max_scaler.inverse_transform(data_imputed), columns = labels_num)
data2 = pd.DataFrame(dataNoNum.values, columns = data.columns.difference(labels_num))

data = data1.join(data2)
data

,Presence,Memory,ReleaseDate,OriginalCost,DiscountedCost,Achievements,Storage,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,...,Web Publishing,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,777456.0,4000.0,2018.000000,39.99,0.00,308.0,50000.000000,198.000000,131.000000,62.000000,...,False,False,False,False,False,False,False,False,False,False
1,158804.0,8000.0,2019.000000,29.99,0.00,0.0,15000.000000,18.000000,4.000000,21.000000,...,False,False,False,False,False,False,False,False,False,False
2,634023.0,4000.0,2016.000000,59.99,0.00,445.0,12000.000000,393.000000,70.000000,219.000000,...,False,False,False,False,False,False,False,False,False,False
3,185610.0,12000.0,2020.000000,59.99,44.99,16.0,14710.503431,16.000000,7.000000,54.000000,...,False,False,False,False,False,False,False,False,True,False
4,130138.0,6000.0,2017.672009,29.99,0.00,90.0,3000.000000,60.000000,6.000000,29.000000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28355,105.0,512.0,2018.000000,0.99,0.00,0.0,10.000000,2.033554,1.000000,1.179881,...,False,False,False,False,False,False,False,False,False,False
28356,13.0,6000.0,2017.000000,0.99,0.00,0.0,2000.000000,1.811846,1.205947,1.000000,...,False,False,False,False,False,False,False,False,False,False
28357,11.0,2048.0,2018.000000,1.99,0.00,0.0,2048.000000,1.500785,1.463039,1.000000,...,False,False,False,False,False,False,False,False,False,False
28358,423.0,500.0,2017.000000,0.99,0.00,0.0,300.000000,1.457396,1.000000,1.000000,...,False,False,False,False,False,False,False,False,False,False


In [19]:
for name in data.columns:
    if data[name].isna().sum() > 0:
        print(name, data[name].isna().sum())

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
#KN = KNeighborsClassifier(n_neighbors=3)
#bool_nulls = ['Controller', 'Indie', 'Soundtrack']
#labels_num = ['Presence','Memory','ReleaseDate','OriginalCost','DiscountedCost','Achievements','Storage',
#                'RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip',
#              'Metacritic','Controller', 'Indie', 'Soundtrack']

#for name in bool_nulls:
#    print(name)
#    bool_nulls.remove(name)
#    target_NoNulls = bool_nulls
#    bool_nulls.append(name)
#    data_aux = data[labels_num].copy()
#    data_aux.dropna(subset=target_NoNulls)
#    
#    Input = data_aux[data_aux[name].notna()]
#    FillValues = data_aux[data_aux[name].isna()]
#    FillValues = FillValues.loc[:, FillValues.columns != name]
#    
#    X_train = Input.loc[:, Input.columns != name]
#    Y_train = Input[name]
#    Y_train = Y_train.astype('int')
    
#    KN.fit(X_train, Y_train)
    
#    data[data[name].isna()][name] = KN.predict(FillValues)
    



In [22]:
for name in data.columns:
    if data[name].isna().sum() > 0:
        print(name, data[name].isna().sum())

In [23]:
for name in labels_num:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

Presence
[777456.  158804.  634023.  ... 226332.   42681.2  45207. ]
Memory
[4.00000000e+03 8.00000000e+03 1.20000000e+04 ... 4.09290443e+03
 2.04204880e+07 9.14959866e+02]
ReleaseDate
[2018.         2019.         2016.         ... 2019.0969449  2017.12776701
 2017.35146751]
OriginalCost
[3.99900000e+01 2.99900000e+01 5.99900000e+01 4.49900000e+01
 1.99900000e+01 1.19900000e+01 2.60210703e+01 1.49900000e+01
 3.49900000e+01 2.20743221e+01 4.99900000e+01 2.49900000e+01
 0.00000000e+00 2.29900000e+01 6.99900000e+01 9.99000000e+00
 2.77761516e+01 5.99000000e+00 6.95495172e+00 4.70589694e-01
 2.50961531e+01 1.79900000e+01 4.99000000e+00 2.99000000e+00
 1.49831505e+01 2.54019301e+01 3.70451688e+01 4.19569181e+00
 7.99000000e+00 1.59900000e+01 6.25225026e+00 6.99000000e+00
 1.69900000e+01 3.78831767e+01 3.19900000e+01 2.39900000e+01
 1.29900000e+01 2.19900000e+01 4.66133869e+00 1.39900000e+01
 3.70614942e+01 7.49000000e+00 2.59900000e+01 5.64931150e+01
 2.28257850e+01 1.46447592e+01 2.6990000

In [24]:
for i in range(len(data['ReleaseDate'])):
    data.loc[i,'ReleaseDate'] = math.floor(data['ReleaseDate'][i])
    
for i in range(len(data['OriginalCost'])):
    data.loc[i,'OriginalCost'] = round(data['OriginalCost'][i], 2)

cols_to_round = ['Achievements', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 
                'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']
for colunm in cols_to_round:
    for i in range(len(data[colunm])):
        data.loc[i,colunm] = round(data[colunm][i])

In [25]:
data.loc[:,'Controller'] = data['Controller'].apply(lambda x: x > 0.5)
data.loc[:,'Indie'] = data['Indie'].apply(lambda x: x > 0.5)
data.loc[:,'Soundtrack'] = data['Soundtrack'].apply(lambda x: x > 0.5)

In [26]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)
    print('Number of na: ' +str(data[name].isna().sum()))

Presence
[777456.  158804.  634023.  ... 226332.   42681.2  45207. ]
Number of na: 0
Memory
[4.00000000e+03 8.00000000e+03 1.20000000e+04 ... 4.09290443e+03
 2.04204880e+07 9.14959866e+02]
Number of na: 0
ReleaseDate
[2018. 2019. 2016. 2020. 2017. 2012. 2015. 2013. 2005. 2014. 2006. 2011.
 2009. 2007. 2008. 1999. 2001. 2010. 1991. 1988. 2004. 1993. 1997. 2003.
 1995. 1987. 1989. 1986. 1990. 2021. 1976. 2000. 1984. 1985. 1982. 1980.
 1979. 2002. 1978. 1983.]
Number of na: 0
OriginalCost
[3.9990e+01 2.9990e+01 5.9990e+01 4.4990e+01 1.9990e+01 1.1990e+01
 2.6020e+01 1.4990e+01 3.4990e+01 2.2070e+01 4.9990e+01 2.4990e+01
 0.0000e+00 2.2990e+01 6.9990e+01 9.9900e+00 2.7780e+01 5.9900e+00
 6.9500e+00 4.7000e-01 2.5100e+01 1.7990e+01 4.9900e+00 2.9900e+00
 1.4980e+01 2.5400e+01 3.7050e+01 4.2000e+00 7.9900e+00 1.5990e+01
 6.2500e+00 6.9900e+00 1.6990e+01 3.7880e+01 3.1990e+01 2.3990e+01
 1.2990e+01 2.1990e+01 4.6600e+00 1.3990e+01 3.7060e+01 7.4900e+00
 2.5990e+01 5.6490e+01 2.2830e+01 1.4640

Number of na: 0
Emotional
[False  True]
Number of na: 0
English
[ True False]
Number of na: 0
Epic
[False  True]
Number of na: 0
Episodic
[False  True]
Number of na: 0
Experience
[False  True]
Number of na: 0
Experimental
[False  True]
Number of na: 0
Exploration
[ True False]
Number of na: 0
FMV
[False  True]
Number of na: 0
FPS
[ True False]
Number of na: 0
Faith
[False  True]
Number of na: 0
Family Friendly
[False  True]
Number of na: 0
Fantasy
[False  True]
Number of na: 0
Farming
[False  True]
Number of na: 0
Farming Sim
[False  True]
Number of na: 0
Fast-Paced
[False  True]
Number of na: 0
Feature Film
[False  True]
Number of na: 0
Female Protagonist
[False  True]
Number of na: 0
Fighting
[False  True]
Number of na: 0
Finnish
[False  True]
Number of na: 0
First-Person
[ True False]
Number of na: 0
Fishing
[False  True]
Number of na: 0
Flight
[False  True]
Number of na: 0
Football
[False  True]
Number of na: 0
Foreign
[False  True]
Number of na: 0
Free to Play
[False  True]
Number

Number of na: 0
Strategy
[False  True]
Number of na: 0
Strategy RPG
[False  True]
Number of na: 0
Stylized
[False  True]
Number of na: 0
Submarine
[False  True]
Number of na: 0
Superhero
[False  True]
Number of na: 0
Supernatural
[False  True]
Number of na: 0
Surreal
[False  True]
Number of na: 0
Survival
[False  True]
Number of na: 0
Survival Horror
[False  True]
Number of na: 0
Swedish
[False  True]
Number of na: 0
Swordplay
[ True False]
Number of na: 0
Tabletop
[False  True]
Number of na: 0
Tactical
[False  True]
Number of na: 0
Tactical RPG
[False  True]
Number of na: 0
Tanks
[False  True]
Number of na: 0
Team-Based
[False  True]
Number of na: 0
Tennis
[False  True]
Number of na: 0
Text-Based
[False  True]
Number of na: 0
Thai
[False  True]
Number of na: 0
Third Person
[False  True]
Number of na: 0
Third-Person Shooter
[False  True]
Number of na: 0
Thriller
[False  True]
Number of na: 0
Time Attack
[False  True]
Number of na: 0
Time Management
[False  True]
Number of na: 0
Time Ma

In [27]:
import matplotlib.pyplot as plt
plt.hist(data['Metacritic'], bins=50)

In [ ]:
import pandas as pd
data['Metacritic'] = pd.qcut(data['Metacritic'], q=4)


In [ ]:
data['Metacritic'].value_counts()

In [ ]:

#def Metacritic_to_Range(x):
#    if x > 75:
#        return "Very good"
    
#    elif x > 50 and x <= 75:
#        return "Good"
    
#    elif x > 25 and x <= 50:
#        return "Bad"
    
#    else:
#        return "Very bad"
        
#actions['Metacritic'].append(Metacritic_to_Range)
#data.loc[:,'Metacritic'] = data['Metacritic'].apply(actions['Metacritic'][0])

In [28]:
rand.seed(random)
x = [y for y in range(len(data))]
x = rand.sample(x, 2200)
data = data.iloc[x]
data

,Presence,Memory,ReleaseDate,OriginalCost,DiscountedCost,Achievements,Storage,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,...,Web Publishing,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,eSports
23342,333.0,512.0,2019.0,1.99,0.0,0.0,200.0,2.0,2.0,2.0,...,False,False,False,False,False,False,False,False,False,False
12545,363.0,1000.0,2016.0,2.99,0.0,0.0,350.0,1.0,1.0,2.0,...,False,False,False,False,False,False,False,False,False,False
27518,102.0,1024.0,2019.0,0.99,0.0,0.0,122.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
19096,269.0,4000.0,2019.0,0.00,0.0,0.0,300.0,2.0,2.0,1.0,...,False,False,False,False,False,False,False,False,False,False
5983,217.0,4000.0,2016.0,11.99,0.0,18.0,4000.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24717,161.0,512.0,2014.0,1.99,0.0,0.0,150.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
19701,411.0,4000.0,2020.0,11.99,0.0,0.0,700.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
26031,249.0,4000.0,2016.0,3.99,0.0,20.0,100.0,1.0,3.0,1.0,...,False,False,False,False,False,False,False,False,False,False
9298,191.0,1000.0,2019.0,14.99,0.0,0.0,800.0,2.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False


In [29]:
data['Metacritic'].value_counts()

In [ ]:
data.to_csv('games_clean.csv', index=False)